In [15]:
# Set up infrastructure and basic problem parameters
import multiprocessing as mp
import numpy as np
import datetime, os
from ContNoRegret.NoRegretAlgos import ContNoRegretProblem
from ContNoRegret.Domains import nBox, UnionOfDisjointnBoxes, DifferenceOfnBoxes, unitbox, hollowbox
from ContNoRegret.LossFunctions import random_PolynomialLosses, random_AffineLosses, random_QuadraticLosses, PolynomialLossFunction
from ContNoRegret.NoRegretAlgos import ContNoRegretProblem
from ContNoRegret.utils import CNR_worker, plot_results, save_results, circular_tour
from ContNoRegret.animate import save_animations
from ContNoRegret.Potentials import (ExponentialPotential, IdentityPotential, pNormPotential, CompositePotential,
                                        ExpPPotential, PExpPotential, HuberPotential, LogtasticPotential, FractionalLinearPotential)

from ContNoRegret.loss_params import *

import matplotlib.pyplot as plt

tmpfolder = '/Volumes/tmp/'


T = len(coeffs) # Time horizon
L = 5.0 # Uniform bound on the Lipschitz constant
N = 2500 # Number of parallel algorithm instances
Ngrid = 250000 # Number of gridpoints for the sampling step

dom = unitbox(3)
nus = [0.05, 1]

# lossfuncs = []
# while len(lossfuncs) < T:
#     tmpfuncs = np.array(random_PolynomialLosses(dom, 10, M, L, 4, [0,1,2,3,4]))
#     normbounds = {nu: np.array([lossfunc.norm(2/nu, tmpfolder=tmpfolder) for lossfunc in tmpfuncs]) for nu in nus}
#     Ms = {nu: np.array(normbounds[nu]) for nu in nus}
#     for i in range(len(normbounds)):
#         if normbounds[nus[0]][i]/normbounds[nus[1]][i] > 5:
#             lossfuncs.append(tmpfuncs[i])
        

lossfuncs = [PolynomialLossFunction(dom, coeff, expo) for coeff,expo in zip(coeffs,exponents)]
Minf, M2 = np.max(inf_norms), np.max(two_norms)

# create Continuous No-Regret problem
prob = ContNoRegretProblem(dom, lossfuncs, L, Minf, desc='normtest')
    
# Select a number of potentials for the Dual Averaging algorithm
potentials = [ExponentialPotential(), pNormPotential(1.05, M=Minf), pNormPotential(2, M=M2)]
  
# the following runs fine if the script is the __main__ method, but crashes when running from ipython
pool = mp.Pool(processes=mp.cpu_count()-1)
processes = []

DAkwargs = [{'opt_rate':True, 'Ngrid':Ngrid, 'potential':pot, 'pid':i, 
             'tmpfolder':tmpfolder, 'label':pot.desc} for i,pot in enumerate(potentials)]
processes += [pool.apply_async(CNR_worker, (prob, N, 'DA'), kwarg) for kwarg in DAkwargs]

# wait for the processes to finish an collect the results
results = [process.get() for process in processes]


Simulating DA, ExpPot (HEDGE), opt. rate eta_t=1.660 sqrt(log t/t)
Simulating DA, pNormPot, $p={1.05}$, opt. rate eta_t=0.216t^(-0.465)$
Simulating DA, pNormPot, $p={2}$, opt. rate eta_t=7.344t^(-0.200)$
pid 0: Starting...
pid 1: Starting...
pid 2: Starting...


KeyboardInterrupt: 